In [3]:
import numpy as np
import pandas as pd

# Logistic Regression Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_function_reg(X, y, weights, lambda_reg):
    m = X.shape[0]
    h = sigmoid(X @ weights)
    epsilon = 1e-5
    reg_term = (lambda_reg / (2 * m)) * np.sum(np.square(weights[1:]))
    cost = -1/m * np.sum(y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon)) + reg_term
    return cost

def gradient_descent_reg(X, y, weights, learning_rate, iterations, lambda_reg):
    m = X.shape[0]
    cost_history = []

    for _ in range(iterations):
        weights[0] -= (learning_rate / m) * np.sum(sigmoid(X @ weights) - y)
        weights[1:] -= (learning_rate / m) * (X[:, 1:].T @ (sigmoid(X @ weights) - y) + lambda_reg * weights[1:])
        cost_history.append(cost_function_reg(X, y, weights, lambda_reg))

    return weights, cost_history

def train_one_vs_rest(X, y, num_classes, learning_rate, iterations, lambda_reg):
    models = []
    m, n = X.shape
    X = np.hstack((np.ones((m, 1)), X))

    for i in range(num_classes):
        y_ovr = np.where(y == i, 1, 0)
        weights = np.zeros(n + 1)
        weights, _ = gradient_descent_reg(X, y_ovr, weights, learning_rate, iterations, lambda_reg)
        models.append(weights)

    return models

def predict(X, models):
    m = X.shape[0]
    X = np.hstack((np.ones((m, 1)), X))
    preds = np.array([sigmoid(X @ w) for w in models]).T
    return np.argmax(preds, axis=1)

# Manual Data Splitting Function
def split_data(data, train_frac, test_frac):
    train_size = int(len(data) * train_frac)
    test_size = int(len(data) * test_frac)
    train_data = data[:train_size]
    validation_data = data[train_size:train_size + test_size]
    test_data = data[train_size + test_size:]
    return train_data, validation_data, test_data

# Manual Feature Scaling Function
def standardize_data(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std, mean, std

# Loading and Preprocessing Data
file_paths = ['./iir1_subset.csv', './iir2_subset.csv', './iir3_subset.csv']
#file_paths = ['./wav1.csv', './wav2.csv', './wav3.csv']
data = []

for i, file_path in enumerate(file_paths):
    df = pd.read_csv(file_path, header=None).T
    df['label'] = i
    data.append(df)

data = pd.concat(data, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)

# Splitting Data
train_data, validation_data, test_data = split_data(data, 0.7, 0.15)
X_train, y_train = train_data.drop('label', axis=1), train_data['label']
X_val, y_val = validation_data.drop('label', axis=1), validation_data['label']
X_test, y_test = test_data.drop('label', axis=1), test_data['label']

# Feature Scaling
X_train_scaled, m, sd= standardize_data(X_train)
X_val_scaled= (X_val-m)/sd#standardize_data(X_val)
X_test_scaled= (X_test-m)/sd#standardize_data(X_test)

# Training Parameters
learning_rate = 0.01
iterations = 3000
num_classes = 3
lambda_reg = 0.1

# Training the Models
models_reg = train_one_vs_rest(X_train_scaled, y_train, num_classes, learning_rate, iterations, lambda_reg)

# Validation and Testing
y_val_pred_reg = predict(X_val_scaled, models_reg)
accuracy_val = np.mean(y_val_pred_reg == y_val)

y_test_pred = predict(X_test_scaled, models_reg)
accuracy_test = np.mean(y_test_pred == y_test)

# Print the accuracy results
print(f"Lamba:{lambda_reg}, Validation Accuracy: {accuracy_val}, Test Accuracy:{accuracy_test}")

print(f"Data mean:{m}")
print(f"Data sd:{sd}")
print(f"Model:{models_reg}")

np.savetxt("Data_mean.csv", m.to_numpy(), delimiter=",")
np.savetxt("Data_sd.csv", sd.to_numpy(), delimiter=",")

pd.DataFrame.from_records(models_reg).to_csv("./Data_model.csv", header=False, index=False)

Lamba:0.1, Validation Accuracy: 0.8546365914786967, Test Accuracy:0.8466833541927409
Data mean:0    0.401803
1    0.546544
2    0.540443
3    0.678557
4    0.677372
5    0.795582
dtype: float64
Data sd:0    0.215647
1    0.300464
2    0.273748
3    0.297058
4    0.326023
5    0.321842
dtype: float64
Model:[array([-1.1161958 , -0.56893064, -1.86121356, -0.29393865,  1.71596618,
       -1.07543646,  1.33522898]), array([-0.89025221,  1.20751163,  0.34137534, -1.09650102, -1.12384217,
        1.41224001, -0.62181464]), array([-1.08744314, -0.82886342,  1.32077476,  1.27968603, -0.55421117,
       -0.30189796, -0.43821752])]


In [4]:
#jupyter labextension install @jupyter-widgets/jupyterlab-manager

import ipywidgets as widgets
from IPython.display import display
from queue import Queue
import pyaudio
from queue import Queue
from threading import Thread
from time import sleep
import numpy as np




messages = Queue()
recordings = Queue()

CHANNELS = 1
FRAME_RATE = 8000 
RECORD_SECONDS = 0.5
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2


record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

audioDevices=[]
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    audioDevices.append((p.get_device_info_by_index(i)['name'],i))
p.terminate()   

deviceList=widgets.Dropdown(
    options=audioDevices,
    value=1,
    description='Audio Device:',
)

volBar=widgets.FloatProgress(
    value=0,
    min=0,
    max=65536,
    description='Volume:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

output = widgets.Output()


def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()



    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=deviceList.value,
                    frames_per_buffer=chunk)
    
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            dt = np.dtype(np.int16)
            dt = dt.newbyteorder('<')
            #v=np.empty()
            for e in frames:
                v=np.frombuffer(e, dtype=dt)
                #v=v.append(np.frombuffer(e, dtype=dt))
                #print(size(v))
                v=np.mean(np.abs(v))
                volBar.value=v
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()
    print("Section done")


def start_recording(data):
    messages.put(True)
    with output:
        print("Starting...")
        print(deviceList.value)
    record = Thread(target=record_microphone)
    record.start()


def stop_recording(data):
    with output:
        messages.get()
        print("Stopped.")


record_button.on_click(start_recording)
stop_button.on_click(stop_recording)


            
display(deviceList,record_button, volBar, stop_button, output)


Dropdown(description='Audio Device:', index=1, options=(('Microsoft Sound Mapper - Input', 0), ('Headset Micro…

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

FloatProgress(value=0.0, bar_style='info', description='Volume:', max=65536.0, style=ProgressStyle(bar_color='…

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

In [ ]:
%pip install pyaudio

In [5]:
%pip install numpy


  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/28/75/3b679b41713bb60e2e8f6e2f87be72c971c9e718b1c17b8f8749240ddca8/numpy-1.26.2-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.2 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 61.2/61.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
    --------------------------------------- 0.2/15.5 MB 4.1 MB/s eta 0:00:04
   - -------------------------------------- 0.6/15.5 MB 6.1 MB/s eta 0:00:03
   -- ------------------------------------- 1.0/15.5 MB 7.4 MB/s eta 0:00:02
   ---- ----------------------------------- 1.6/15.5 MB 8.4 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/15.5 MB 8.9 MB/s eta 0:00:02
   ------ --------------------------------- 2.6/15.5 MB 9.1 MB/s eta 0:00:02
   ------- -------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
